In [1]:
#Dependencies

#K-means algorithm (finding K method 1)
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#PCA finding K (finding K method 2)
from sklearn.decomposition import PCA

#dependencies for K-Nearest Neighbors (KNN) Machine learning
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
pd.set_option('display.max_columns', None)

#each time it says a standalone "df", plz replace w the dataframe we are using

In [2]:
df = pd.read_csv("C:/Users/lnovo_crisissupport/Desktop/SMU Data Analytics BC/Project-4/Analysis Components/Resource/BankChurners.csv")

# Drop duplicates and columns from the DataFrame
df.drop_duplicates()
columns_to_drop = ['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1']
credit_df = df.drop(columns_to_drop, axis=1)

# Replace values in the 'Attrition_Flag' column
credit_df['Attrition_Flag'] = credit_df['Attrition_Flag'].replace({'Attrited Customer': 0, 'Existing Customer': 1})

In [3]:
# Perform groupBy and count operation on 'Attrition_Flag' column
attrition_flag_counts = credit_df['Attrition_Flag'].value_counts().reset_index()
attrition_flag_counts.columns = ['Attrition_Flag', 'count']

# Join the original DataFrame with the DataFrame containing counts
updated_df = credit_df.merge(attrition_flag_counts, on='Attrition_Flag', how='left')

df_8500 = updated_df[updated_df["count"]==8500]
df_sample = df_8500.sample(frac=1627.0/8500.0, random_state=1)

df_1627 = updated_df[updated_df["count"] !=8500]

# Combine the sampled data with the original data
updated_df = pd.concat([df_sample, df_1627])

updated_df.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,count
3263,1,59,F,2,High School,Single,Unknown,Blue,52,4,2,4,3578.0,663,2915.0,0.674,4065,79,0.646,0.185,8500
2936,1,31,F,0,College,Married,Less than $40K,Blue,21,5,2,2,1677.0,0,1677.0,1.369,3059,50,0.613,0.000,8500
1683,1,26,M,0,College,Single,$40K - $60K,Blue,13,5,2,2,2047.0,1196,851.0,0.623,2180,33,0.941,0.584,8500
6208,1,55,F,1,Graduate,Married,Less than $40K,Blue,46,3,3,3,1606.0,0,1606.0,0.855,4548,76,0.652,0.000,8500
2559,1,34,M,1,Unknown,Married,$60K - $80K,Blue,29,3,2,3,3890.0,2286,1604.0,0.633,1889,51,0.457,0.588,8500


In [ ]:
credit_df

In [4]:
#replacing 'Attrition_Flag' values
credit_df['Attrition_Flag'] = credit_df['Attrition_Flag'].replace({'False': 0, 'True': 1})

# Identify non-numeric columns
non_numeric_cols = credit_df.select_dtypes(include=['object']).columns

# Iterate over non-numeric columns to replace or drop
for col in non_numeric_cols:
    # Example replacement for specific columns (customize as needed)
    if col == 'SomeCategoricalColumn':  # Replace with your actual column name
        credit_df[col] = credit_df[col].replace({'Category1': 0, 'Category2': 1, 'Category3': 2})
    else:
        # Drop non-numeric columns
        credit_df = credit_df.drop(col, axis=1)

# Ensure all data is now numeric
assert credit_df.select_dtypes(include=['object']).empty, "There are still non-numeric columns in the DataFrame"

In [5]:
#replacing 'Attrition_Flag' values
updated_df['Attrition_Flag'] = updated_df['Attrition_Flag'].replace({'False': 0, 'True': 1})

# Identify non-numeric columns
UDnon_numeric_cols = updated_df.select_dtypes(include=['object']).columns

# Iterate over non-numeric columns to replace or drop
for col in UDnon_numeric_cols:
    # Example replacement for specific columns (customize as needed)
    if col == 'SomeCategoricalColumn':  # Replace with your actual column name
        updated_df[col] = updated_df[col].replace({'Category1': 0, 'Category2': 1, 'Category3': 2})
    else:
        # Drop non-numeric columns
        updated_df = updated_df.drop(col, axis=1)

# Ensure all data is now numeric
assert updated_df.select_dtypes(include=['object']).empty, "There are still non-numeric columns in the DataFrame"

In [10]:
#K-means algorithm (unbalanced)
#identifies clusters from a multidimensional standpt, which isn't totally linear and can point in the direction of influencing variables

#create a list for inertia and values of K
inertia = []
k = list(range(1, 5))

#for loop for K-means algorithm
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(credit_df)
    inertia.append(k_model.inertia_)

#dictionary (variable) for elbow method data
elbowK_data = {"k": k, "inertia": inertia}
    #convert to df named df_elbowK
df_elbowK = pd.DataFrame(elbowK_data)

#plot it to find the best K - visually look at the plot and find K with the elbow method
df_elbowK.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k,
    width=300,  
    height=150,
    fontsize={"title": 16, "labels": 12}
)

:Curve   [k]   (inertia)

In [11]:
#K-means algorithm (balanced)
#identifies clusters from a multidimensional standpt, which isn't totally linear and can point in the direction of influencing variables

#create a list for inertia and values of K
inertia = []
k = list(range(1, 5))

#for loop for K-means algorithm
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(updated_df)
    inertia.append(k_model.inertia_)

#dictionary (variable) for elbow method data
elbowK_data = {"k": k, "inertia": inertia}
    #convert to df named df_elbowK
df_elbowK = pd.DataFrame(elbowK_data)

#plot it to find the best K - visually look at the plot and find K with the elbow method
df_elbowK.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k,
    width=300,  
    height=150,
    fontsize={"title": 16, "labels": 12}
)

:Curve   [k]   (inertia)

In [12]:
#PCA - turning multidimensional into a 2-dimensional analysis here.
#reduces dimensionality overall to show a linear (2d) relationship btwn variables

pca = PCA(n_components=2)
rename_this_PCA = pca.fit_transform(credit_df)

#turn it from an array into a df
newPCAdf = pd.DataFrame(
    rename_this_PCA,
    columns=["PCA1", "PCA2"]
)

#store inertia values and set the min and max k clusters
inertia = []
k = list(range(1, 5))

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(newPCAdf)
    inertia.append(k_model.inertia_)

#turn it into a df named df_elbowPCA
elbowPCA_data = {"k": k, "inertia": inertia}
df_elbowPCA = pd.DataFrame(elbowPCA_data)

#plot it to find the best K - visually look at the plot and find K with the elbow method
df_elbowPCA.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k,
    width=300,  
    height=150,
    fontsize={"title": 16, "labels": 12}
)

:Curve   [k]   (inertia)

In [13]:
#PCA - turning multidimensional into a 2-dimensional analysis here.
#reduces dimensionality overall to show a linear (2d) relationship btwn variables

pca = PCA(n_components=2)
rename_this_PCA = pca.fit_transform(updated_df)

#turn it from an array into a df
newPCAdf = pd.DataFrame(
    rename_this_PCA,
    columns=["PCA1", "PCA2"]
)

#store inertia values and set the min and max k clusters
inertia = []
k = list(range(1, 5))

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(newPCAdf)
    inertia.append(k_model.inertia_)

#turn it into a df named df_elbowPCA
elbowPCA_data = {"k": k, "inertia": inertia}
df_elbowPCA = pd.DataFrame(elbowPCA_data)

#plot it to find the best K - visually look at the plot and find K with the elbow method
df_elbowPCA.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k,
    width=300,  
    height=150,
    fontsize={"title": 16, "labels": 12}
)

:Curve   [k]   (inertia)

In [ ]:
#Compare the K value plots
#K-means algorithm: # of groups with all dimensions considered
#PCA: # of groups with dimensions boiled down to 2 for 2d grouping (lil more vague)

In [ ]:
#KNN Machine Learning
#algorithm stores the entire training dataset in memory

    #Predicts by
#Calculating dif btwn the new data point and all points in the training dataset
#Identifies K nearest neighbors (data points) to the new data point based on distance
#Predicts new data point by taking the average or weighted average of the values of its K nearest neighbors

In [14]:
#KNN Machine Learning pt 1

#isolate Attrition status (flag?) as y variable
y = credit_df['Attrition_Flag']
X = credit_df.drop(columns='Attrition_Flag')
    #convert categorical x variables to binary
X = pd.get_dummies(X)

X.head()


,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,49,5,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,51,3,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,40,4,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,40,3,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [15]:
#set training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#scale the (now all) numerical values so there aren't extremes, easy computation
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
    #SET K VALUE FOUND FROM ABOVE
model = KNeighborsClassifier(n_neighbors=3)

#train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [27]:
#KNN Machine Learning pt 2

#make predictions
y_pred = model.predict(X_test_scaled)
#print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_pred,y_test))
print()

#Classification Report
    #Precision = accuracy of positive predictions made by the model. 
        #"Of all the instances predicted as positive, how many are actually positive?"
        #high precision = low false positive rate (less false alarms)
    #Recall/sensitivity/true positive rate (TPR) = ability to correctly identify positive instances from all actual positive instances in the dataset.
        #"Of all the actual positive instances, how many did the model correctly identify?" 
        #high recall = low false negative rate 
        #F-1 = harmonic mean of precision and recall, balances both precision and recall (overall rating of model performance)
print("Classification Report:")
print(classification_report(y_pred,y_test))

Confusion Matrix:
[[ 268   64]
 [ 137 2063]]

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.81      0.73       332
           1       0.97      0.94      0.95      2200

    accuracy                           0.92      2532
   macro avg       0.82      0.87      0.84      2532
weighted avg       0.93      0.92      0.92      2532



In [28]:
#KNN Machine Learning pt 1 (balanced)

#isolate Attrition status (flag?) as y variable
y = updated_df['Attrition_Flag']
X = updated_df.drop(columns='Attrition_Flag')
    #convert categorical x variables to binary
X = pd.get_dummies(X)

#set training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#scale the (now all) numerical values so there aren't extremes, easy computation
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
    #SET K VALUE FOUND FROM ABOVE
model = KNeighborsClassifier(n_neighbors=3)

#train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [29]:
#KNN Machine Learning pt 2 (balanced)

#make predictions
y_pred = model.predict(X_test_scaled)
#print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_pred,y_test))
print()

#Classification Report
    #Precision = accuracy of positive predictions made by the model. 
        #"Of all the instances predicted as positive, how many are actually positive?"
        #high precision = low false positive rate (less false alarms)
    #Recall/sensitivity/true positive rate (TPR) = ability to correctly identify positive instances from all actual positive instances in the dataset.
        #"Of all the actual positive instances, how many did the model correctly identify?" 
        #high recall = low false negative rate 
        #F-1 = harmonic mean of precision and recall, balances both precision and recall (overall rating of model performance)
print("Classification Report:")
print(classification_report(y_pred,y_test))

Confusion Matrix:
[[403   0]
 [  2 409]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       403
           1       1.00      1.00      1.00       411

    accuracy                           1.00       814
   macro avg       1.00      1.00      1.00       814
weighted avg       1.00      1.00      1.00       814

